In [2]:
from gtts import gTTS
import os


sesler = {
    "baslat": "Görme engelli yol asistanı başlatıldı",
    "kapat": "Asistan kapatıldı",
    "engel": "Önünüzde bir engel var",
    
    # Nesneler
    "kisi": "Önünüzde bir kişi var",
    "araba": "Önünüzde bir araba var",
    "sandalye": "Önünüzde bir sandalye var",
    "sise": "Önünüzde bir şişe var",
    "bardak": "Önünüzde bir bardak var",
    "telefon": "Önünüzde bir telefon var",
    "bilgisayar": "Önünüzde bir bilgisayar var",
    "canta": "Önünüzde bir çanta var",
    "masa": "Önünüzde bir masa var",
    "koltuk": "Önünüzde bir koltuk var"
}

print("Ses dosyaları oluşturuluyor, lütfen bekleyin...")

for isim, metin in sesler.items():
    tts = gTTS(text=metin, lang="tr")
    tts.save(f"{isim}.mp3")
    print(f"✅ {isim}.mp3 oluşturuldu.")

print("Tüm ses dosyaları hazır!")


Ses dosyaları oluşturuluyor, lütfen bekleyin...
✅ baslat.mp3 oluşturuldu.
✅ kapat.mp3 oluşturuldu.
✅ engel.mp3 oluşturuldu.
✅ kisi.mp3 oluşturuldu.
✅ araba.mp3 oluşturuldu.
✅ sandalye.mp3 oluşturuldu.
✅ sise.mp3 oluşturuldu.
✅ bardak.mp3 oluşturuldu.
✅ telefon.mp3 oluşturuldu.
✅ bilgisayar.mp3 oluşturuldu.
✅ canta.mp3 oluşturuldu.
✅ masa.mp3 oluşturuldu.
✅ koltuk.mp3 oluşturuldu.
Tüm ses dosyaları hazır!


In [ ]:
import cv2
import time
import numpy as np
from ultralytics import YOLO
from pygame import mixer
import threading



mixer.init()

def sesli_bildir(dosya):
    """Non-blocking mp3 çalma - Eğer zaten çalıyorsa bölmez"""
    if mixer.music.get_busy():
        return 
        
    def cal():
        try:
            mixer.music.load(dosya)
            mixer.music.play()
        except Exception as e:
            print(f"Ses çalınamadı ({dosya}): {e}")
            
    threading.Thread(target=cal, daemon=True).start()


ceviriler = {
    "person": "kisi",
    "chair": "sandalye",
    "car": "araba",
    "bottle": "sise",
    "cup": "bardak",
    "cell phone": "telefon",
    "laptop": "bilgisayar",
    "backpack": "canta",
    "dining table": "masa",
    "couch": "koltuk",
    "tv": "televizyon",   
    "mouse": "fare"       
}


def engel_var_mi(frame):
    gri = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gri, (7,7), 0)
    edges = cv2.Canny(blur, 50, 150)

    h, w = edges.shape

    merkez = edges[int(h*0.30):int(h*0.80), int(w*0.20):int(w*0.80)]
    
    kenar_orani = np.sum(merkez > 0) / merkez.size
    
  
    return kenar_orani > 0.05 


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Model yükleniyor...")
model = YOLO("yolov8s.pt")

sesli_bildir("baslat.mp3")


son_uyari = 0
frame_sayaci = 0
son_sonuc = None
karanlik_sayac = 0

print("Sistem Aktif! Çıkmak için 'q' tuşuna bas.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_sayaci += 1
    simdi = time.time()

   
    if np.mean(frame) < 10:
        karanlik_sayac += 1
    else:
        karanlik_sayac = 0

    if karanlik_sayac > 90: # ~3 saniye karanlık
        sesli_bildir("kapat.mp3")
        print("Karanlık nedeniyle kapatıldı.")
        break


    if frame_sayaci % 5 == 0:
        results = model(frame, conf=0.45, verbose=False)
        if results:
            son_sonuc = results[0]


    annotated = frame.copy()
    tespit_edilen_kodlar = [] 

    if son_sonuc and son_sonuc.boxes:
        annotated = son_sonuc.plot()
        for box in son_sonuc.boxes:
            cls_id = int(box.cls[0])
            ingilizce_isim = model.names[cls_id]
            
        
            if ingilizce_isim in ceviriler:
                tespit_edilen_kodlar.append(ceviriler[ingilizce_isim])

    
    if simdi - son_uyari > 4: 
        
        
        if tespit_edilen_kodlar:
           
            algilanan_nesne = tespit_edilen_kodlar[0] 
            
            dosya_adi = f"{algilanan_nesne}.mp3"
            print(f"Tanındı: {algilanan_nesne} -> Çalınıyor: {dosya_adi}")
            
            sesli_bildir(dosya_adi)
            son_uyari = simdi
            
     
        elif engel_var_mi(frame):
            print("Tanımsız Engel Algılandı!")
            sesli_bildir("engel.mp3")
            son_uyari = simdi

    cv2.imshow("Gorme Engelli Yol Asistani", annotated)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:
        sesli_bildir("kapat.mp3")
        time.sleep(2)
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\kubradur\anaconda3\envs\env_bst4\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.9.23)
Hello from the pygame community. https://www.pygame.org/contribute.html
Model yükleniyor...
Sistem Aktif! Çıkmak için 'q' tuşuna bas.
Tanındı: kisi -> Çalınıyor: kisi.mp3
Tanındı: televizyon -> Çalınıyor: televizyon.mp3
Ses çalınamadı (televizyon.mp3): No file 'televizyon.mp3' found in working directory 'C:\Users\kubradur'.
Tanındı: bilgisayar -> Çalınıyor: bilgisayar.mp3
Tanındı: sise -> Çalınıyor: sise.mp3
